In [1]:
# !pip install -q langchain openai tiktoken chromadb

In [2]:
# !wget -q https://github.com/kairess/toy-datasets/raw/master/techcrunch-articles.zip
# !unzip -q techcrunch-articles.zip -d articles

In [13]:
import os

os.environ["OPENAI_API_KEY"] = ""


In [4]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader

In [5]:
# loader = TextLoader('single_text_file.txt')
# loader = DirectoryLoader('./articles', glob="*.txt", loader_cls=TextLoader)
# loader = DirectoryLoader('./articles', glob="*.txt", loader_cls=TextLoader)
loader = DirectoryLoader('./articles', glob="*.pdf", loader_cls=PyPDFLoader)

documents = loader.load()

len(documents)

2002

In [6]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# 1000글자씩 분할하기
# 끊기는것을 방지하지 위해 겹치는 부분을 200자로 제한함.
texts = text_splitter.split_documents(documents)

len(texts)

2424

In [7]:
texts[2:4]

[Document(page_content='70.7%\n12.6%\n6.3%10.4%37.8%\n18.2%\n9.4%34.6%\n귀농 귀촌\n1\n2\n3\n4\n5\n6', metadata={'source': 'articles\\(농업정책관-청년농육성정책팀) 2022년 귀농귀촌실태조사 결과 보도자료(3.3. 조간)_별첨(2022년 귀농귀촌실태조사 결과  인포그래픽).pdf', 'page': 0}),
 Document(page_content='∙\n일반직장\n취업자영업 임시직 농사일 비농업부문\n일용직농업임금노동\n6.4% 83.3%귀농 귀촌 (만원)\n2,3183,099 3,055 3,1483,186 3,2063,4543,6853,877 3,929 3,8084,045\n귀농, 귀촌첫해\n(전체평균)1년차 2년차 3년차\n 5년차\n57.4%\n22.7%\n7.4% 5.0% 3.1% 2.6%\n귀농\n 귀촌\n∙', metadata={'source': 'articles\\(농업정책관-청년농육성정책팀) 2022년 귀농귀촌실태조사 결과 보도자료(3.3. 조간)_별첨(2022년 귀농귀촌실태조사 결과  인포그래픽).pdf', 'page': 1})]

## 2. Create Chroma DB

In [8]:
# persist_directory = 'db' # 'db' 디렉토리에 저장함.

# embedding = OpenAIEmbeddings()
# # openai의 embedding을 사용함

# vectordb = Chroma.from_documents(
#     documents=texts, 
#     embedding=embedding,
#     persist_directory=persist_directory)

# 임베딩 모델 로드
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(
    model_name="jhgan/ko-sroberta-multitask", encode_kwargs={'normalize_embeddings': True}
)

# Chroma DB에 벡터화하여 저장하기
from langchain.vectorstores import Chroma
vectordb = Chroma.from_documents(texts, embeddings, persist_directory="./chroma_db")


In [9]:
# 초기화 하는 하는 과정
vectordb.persist()
vectordb = None
persist_directory = 'chroma_db'

In [10]:
vectordb = Chroma(
    persist_directory=persist_directory, 
    embedding_function=embeddings)

## 3. Make a retriever

In [11]:
retriever = vectordb.as_retriever()
# docs = retriever.get_relevant_documents("What is Generative AI?")

# for doc in docs:
#     print(doc.metadata["source"])

## 4. Make a chain

In [17]:
from langchain.chat_models import ChatOpenAI
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model_name = "gpt-3.5-turbo", temperature = 0), 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True)

e:\Installation\Anaconda3\envs\py311_gpu\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [18]:
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

## 5. Query

In [20]:
# query = "벼 농사가 망했을때 보상받을 수 있는 방법에 대해 알려줘"
# query = "사업자 등록을 위해 구비해야할 서류는 뭐가 있을까?"
query = "사과 재배하는 방법과, 사과 품질을 관리하는 방법을 알려줘"
llm_response = qa_chain(query)
process_llm_response(llm_response)

사과 재배하는 방법은 다음과 같습니다:

1. 품종 선택: 사과 품종은 재배 환경과 시장 요구에 맞게 선택해야 합니다. 품종은 수확 시기, 내한성, 병해충 저항성, 과실 품질 등을 고려하여 선택해야 합니다.

2. 묘목 구입: 좋은 품질의 사과 묘목을 구입해야 합니다. 묘목은 제뿌리 대목이고, 뿌리가 충실하며, 묘목의 굵기와 길이 등이 기준에 맞아야 합니다.

3. 접목: 사과는 주로 접목에 의해 묘목을 생산합니다. 접목은 대목과 접목묘를 결합시키는 작업으로, 접목 시기와 방법을 잘 따라야 합니다.

4. 토양 관리: 사과는 토양의 영양분과 수분을 필요로 합니다. 토양의 pH 조절, 영양분 공급, 수분 관리 등을 통해 사과나무가 건강하게 자라도록 해야 합니다.

5. 병해충 관리: 사과는 다양한 병해충에 취약합니다. 병해충의 발생을 예방하기 위해 적기에 살충제를 사용하고, 병해충의 피해를 최소화하기 위해 정기적인 관찰과 조치가 필요합니다.

사과 품질을 관리하는 방법은 다음과 같습니다:

1. 결실 관리: 사과의 결실을 관리하여 과실의 크기와 품질을 향상시킬 수 있습니다. 적절한 간격으로 과실을 희석하고, 과실의 크기와 형태를 조절하는 등의 관리 작업이 필요합니다.

2. 병해충 관리: 병해충의 피해로 인해 사과의 품질이 저하될 수 있습니다. 정기적인 병해충 관찰과 적기에 살충제를 사용하여 병해충의 피해를 최소화해야 합니다.

3. 수확 시기: 사과의 수확 시기는 과실의 성숙도와 품질에 영향을 미칩니다. 적절한 수확 시기를 선정하여 과실의 당도와 신선도를 유지해야 합니다.

4. 보관 관리: 사과는 장기 보관이 가능한 과일이지만, 적절한 보관 조건과 기간을 유지해야 합니다. 온도와 습도를 조절하여 사과의 신선도와 품질을 유지할 수 있습니다.

이러한 사과 재배와 품질 관리 방법을 지속적으로 실천하면, 좋은 품질의 사과를 생산할 수 있습니다.


Sources:
articles\농업기술길잡이5_사과재배.PDF
articles\농업기술길잡이5_사과재배.PDF
ar

In [ ]:
llm_response

{'query': '사업자 등록을 위해 구비해야할 서류는 뭐가 있을까?',
 'result': '\n사업자등록신청서, 사업허가증, 사업계획서, 임대차계약서, 동업계약서, 도면, 자금출처명세서, 허가(신고, 등록)증 사본 등이 필요합니다.',
 'source_documents': [Document(page_content='후계농 나특별 사장님의  후계농 나특별 사장님의  \n사업자등록 안내사업자등록 안내\n농업인이 인터넷으로 직접 농산물을 판매하고, \n신용카드 결제· 세금계산서 발급 등을 위해서는 \n사업자등록을 하여야 합니다. 농업인(개인사업자)이 \n영농조합법인· 농업회사법인(법인사업자)으로  \n전환하는 경우에도 사업자등록이 이루어져야 합니다.  \n사업자는 연 1회 수입금액 명세 등을 포함한 사업자 \n현황을 세무서에 신고해야 하며, 사업자등록은 \n사업 개시일로부터 20일 이내에 다음의 구비서류를 \n갖추어 사업장 관할 세무서장에게 신청하면 됩니다.\n사업을 시작할 때는 \n반드시 사업자등록구비서류\n•사업자등록신청서 1부\n• 사업허가증·등록증 또는 신고확인증 사본 1부  \n(허가를 받거나 등록 또는 신고를 하여야 하는 사업의 경우)\n• 사업허가신청서 사본이나 사업계획서  \n(허가 전에 등록을 하고자 하는 경우)\n•임대차계약서 사본 1부 (사업장을 임차한 경우)\n• 2인 이상 공동으로 사업을 하는 경우에는 동업계약서 등  \n공동사업을 증명할 수 있는 서류  \n(사업자등록은 공동사업자 중 1인을 대표로 하여 신청)\n•도면 1부 (상가건물임대차보호법이 적용되는 건물의 일부를 임차한 경우)\n• 자금출처 명세서 1부 (금자금 도소매업, 과세유흥장소 영위자,  \n연료판매업, 재생용 재료수집·판매업의 경우 등)\n축산업, 축산물 가공업 등 관할관청의 허가, 신고,  \n등록대상 업종인 경우에는 사업자등록 신청시  \n허가(신고, 등록)증 사본을 제출하여야 합니다.\n단, 허가(신고, 등록) 전에 사업자등록신청을 하는 경우에는 허가\n(등록)신